# 50天測試

In [2]:
C_test_1_data = pd.read_excel('A(塑化)_50天_ques.xlsx')

NameError: name 'pd' is not defined

In [359]:
name= [ ' - MARKET VALUE', ' - DIVIDEND YIELD', ' - PER',
       ' - PRICE INDEX', ' - ADJUSTMENTFACTOR', ' - DIV.PER SHR.',
       ' - EARNINGS PER SHR', ' - HISTORICAL VOLATILITY 5 YEARS',
       ' - MARKET VAL BY CO.', ' - ASK PRICE', ' - BID PRICE', ' - PRICE HIGH',
       ' - PRICE LOW', ' - OPENING PRICE', ' - RETURN IND AS PAID',
       ' - TOT RETURN IND', ' - TURNOVER BY VALUE', ' - TURNOVER BY VOLUME',
       ' - UNADJ. PRICE OPEN', ' - UNADJUSTED PRICE', ' - UNADJ. ASK PRICE',
       ' - UNADJ. BID PRICE', ' - UNADJ. PRICE HIGH', ' - UNADJ. PRICE LOW',
       ' - UNADJ. VOLUME', ' - VOL WGHTD AV PRICE']
C_test_1_data = C_test_1_data[name]
feature_columns = list(A_test_1_data.columns)


In [362]:
A_test_1_data=A_test_1_data.fillna(method='ffill')
A_test_1_data.fillna(method='bfill',inplace=True)

In [363]:
classi = []
for row in range(len(A_test_1_data)):
    try:
        if ((A_test_1_data.iloc[row+1,13]-A_test_1_data.iloc[row,13])/A_test_1_data.iloc[row,13]) > 0:
            classi.append(1)
        elif((A_test_1_data.iloc[row+1,13]-A_test_1_data.iloc[row,13])/A_test_1_data.iloc[row,13]) == 0:
            classi.append(0)
        else:
            classi.append(-1)
    except:
        classi.append(99)
A_test_1_data['三元'] = classi
A_test_1_data.三元.value_counts()

 1     21
-1     20
 0      8
 99     1
Name: 三元, dtype: int64

In [365]:
test = A_test_1_data
# Target column
#target_adj_close = pd.DataFrame(test['股價'])
display(test.head())

,- MARKET VALUE,- DIVIDEND YIELD,- PER,- PRICE INDEX,- ADJUSTMENTFACTOR,- DIV.PER SHR.,- EARNINGS PER SHR,- HISTORICAL VOLATILITY 5 YEARS,- MARKET VAL BY CO.,- ASK PRICE,...,- TURNOVER BY VOLUME,- UNADJ. PRICE OPEN,- UNADJUSTED PRICE,- UNADJ. ASK PRICE,- UNADJ. BID PRICE,- UNADJ. PRICE HIGH,- UNADJ. PRICE LOW,- UNADJ. VOLUME,- VOL WGHTD AV PRICE,三元
2233,605914.4,6.68,9.4,907.8,1,5.1,8.17,0.1803,605914.4,76.4,...,4492.8,76.0,76.4,76.4,76.3,76.5,75.7,4492.8,76.05,-1
2234,596397.5,6.78,9.2,893.5,1,5.1,8.17,0.1807,596397.5,75.3,...,6338.8,75.6,75.2,75.3,75.2,75.8,75.2,6338.8,75.44,-1
2235,597983.6,6.76,9.2,895.9,1,5.1,8.17,0.1811,597983.6,75.4,...,5431.4,75.0,75.4,75.4,75.3,76.1,74.5,5431.4,75.27,-1
2236,586880.6,6.89,9.1,879.3,1,5.1,8.17,0.1816,586880.6,74.1,...,7523.7,74.0,74.0,74.1,74.0,74.8,73.8,7523.7,74.26,1
2237,595604.5,6.79,9.2,892.3,1,5.1,8.17,0.1819,595604.5,75.1,...,4935.8,74.5,75.1,75.1,75.0,75.2,74.0,4935.8,74.67,1


In [ ]:
from sklearn.preprocessing import MinMaxScaler
feature_columns = list(A_test_1_data.columns)

scaler = MinMaxScaler()
feature_minmax_transform_data = scaler.fit_transform(test[feature_columns])
feature_minmax_transform = pd.DataFrame(columns=feature_columns, data=feature_minmax_transform_data, index=test.index)
feature_minmax_transform.head()

X_test =np.array(feature_minmax_transform.values)
X_tst_t = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [372]:
#y_test = A_test_1_data.iloc[:,0].values
y_pred_test_lstm = model_lstm.predict(X_tst_t)
y_50ds_pred= [ i[0] for i in y_pred_test_lstm]
#print("The R2 score on the Test set is:\t{:0.3f}".format(r2_score(y_test, y_pred_test_lstm)))
#r2_test = r2_score(y_test, y_pred_test_lstm)

In [1]:
mm = -0.000197744
std = 0.009412714

table_pred = []
y_50ds_return = []
for i in range(len(y_50ds_pred)):
    try:
        y_50ds_return.append((y_50ds_pred[i+1]-y_50ds_pred[i])/y_50ds_pred[i])
        if (y_50ds_pred[i+1]-y_50ds_pred[i])/y_50ds_pred[i] > mm+std:
            table_pred.append(1)
        elif (y_50ds_pred[i+1]-y_50ds_pred[i])/y_50ds_pred[i] < mm-std:
            table_pred.append(-1)
        else:
            table_pred.append(0)
    except:
        table_pred.append(99)
        y_50ds_return.append(99)

pd.DataFrame({'三元':table_pred,'預測值':y_50ds_pred,'報酬率':y_50ds_return}).to_csv('123[測試]50dys_C.csv')

NameError: name 'y_50ds_pred' is not defined